In [1]:
import pandas as pd
import numpy as np

In [2]:
TEST_SET='../../datasets/mimic-iii-notes/NOTES_len6K_src5900_cov0.5_w_ESGs_test_split.jsonl'
PREDICTION_SET='../../evaluation/clinical/prediction_output/20240317115909/predictions.jsonl'
METRIC_SET='../../evaluation/clinical/prediction_output/20240317115909/metric_scores.jsonl'

In [3]:
test_set = pd.read_json(TEST_SET, lines=True)
pred_set = pd.read_json(PREDICTION_SET, lines=True)
metric_set = pd.read_json(METRIC_SET, lines=True)

test_set = test_set.rename(columns={'example_id': 'id'})

for df in (test_set, pred_set, metric_set):
    print(df.columns)

Index(['id', 'source', 'target', 'len_source', 'len_target', 'len_total',
       'label_entity_coverage_rate', 'source_orig', 'len_src_tok',
       'len_tgt_tok', 'len_total_tok', 'oracle_salient_ESGs'],
      dtype='object')
Index(['id', 'output'], dtype='object')
Index(['id', 'SGR', 'HR', 'rouge-1%', 'rouge-2%', 'rouge-l%', 'bleu-4',
       'bs_f1', 'bs_p'],
      dtype='object')


In [4]:
def three_table_inner_join(df1, df2, df3, join_key):
    return pd.merge(
                pd.merge(
                    df1,
                    df2, on=join_key, how='inner'),
                df3, on=join_key, how='inner')

In [5]:
result_set = three_table_inner_join(test_set, pred_set, metric_set, join_key='id')

In [6]:
assert test_set.shape[0] == pred_set.shape[0] == metric_set.shape[0] == result_set.shape[0]

In [7]:
result_set

,id,source,target,len_source,len_target,len_total,label_entity_coverage_rate,source_orig,len_src_tok,len_tgt_tok,...,oracle_salient_ESGs,output,SGR,HR,rouge-1%,rouge-2%,rouge-l%,bleu-4,bs_f1,bs_p
0,19322119378,Sinus rhythm. {{Low limb lead voltage}}. {{Pri...,Patient is a 62 year-old male with metastatic ...,10092,2859,12951,0.579710,Sinus rhythm. Low limb lead voltage. Prior ant...,3585,826,...,"[[Heparinized saline, pericardial drainage, fu...",62 yo male with RCC and pericardial effusion s...,1.0,0.142857,17.2185,4.0323,10.3524,9.6668,0.273645,0.305546
1,20091116425,Sinus rhythm Possible {{anterior infarct}} - a...,Endocrine: The patient was initially admitted ...,13732,5580,19312,0.670330,Sinus rhythm Possible anterior infarct - age u...,4683,1439,...,"[[Right ventricular systolic function, Sinus b...","63 yo F with history of hypertension, hyperlip...",0.0,1.000000,12.4731,4.1284,12.6997,17.6941,0.239375,0.306435
2,12169107757,"Sinus rhythm, without {{diagnostic abnormality...",A P: 71 year old female with multiple medical ...,13436,2363,15799,0.581818,"Sinus rhythm, without diagnostic abnormality. ...",4936,683,...,"[[NPO, BS, ST depression, 2l NC, Left atrial a...",{{Gastrointestinal bleeding}}: The patient was...,0.0,1.000000,28.6645,8.8632,15.6216,17.7464,0.413937,0.433685
3,19738102287,Sinus rhythm. {{Left ventricular hypertrophy}}...,The patient was admitted to the Medical Intens...,7206,3546,10752,0.666667,Sinus rhythm. Left ventricular hypertrophy. Co...,2794,962,...,"[[difficulty, similar ekg changes, Recurrent l...",This is a 72 year old man with a history of co...,1.0,0.941176,13.8801,4.3919,10.7774,13.6782,0.297437,0.375215
4,7758102646,2:07 PM CHEST PORTABLE AP Clip # Reason: {{che...,Upper gastrointestinal bleed: Because of his a...,10931,4303,15234,0.625000,2:07 PM CHEST PORTABLE AP Clip # Reason: check...,4191,1160,...,"[[The pulmonary artery systolic pressure, UPPE...","56 year old male with history of colon cancer,...",1.0,0.714286,16.4306,5.0671,17.1546,26.0257,0.331953,0.410842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,77815103081,10:53 AM CAROT CEREB Clip # Reason: {{Stent as...,The patient is a 58 year old man who was elect...,3608,630,4238,0.555556,10:53 AM CAROT CEREB Clip # Reason: Stent assi...,1126,168,...,"[[a 6 French Angio-Seal closure device, A Neur...",{{Stent assisted coiling of aneurysmAnesthesia...,1.0,0.450000,19.1919,2.4390,11.8681,24.7803,0.267703,0.206402
262,30660107310,{{Baseline artifact}}. Sinus rhythm. {{Low lim...,60 yo F longtime smoker with h o COPD and schi...,14559,1129,15688,0.700000,Baseline artifact. Sinus rhythm. Low limb lead...,5542,328,...,"[[O2 SAT, VERY SOB, NPO, mask ventilation, BS,...",60 yo F with schizoaffective disorder and long...,1.0,0.529412,28.4211,9.8940,25.3602,39.9881,0.423235,0.486041
263,31245109216,Patient admitted from OSH in NY. She has PMH o...,The patient was admitted for stent replacement...,15159,1785,16944,0.761905,Patient admitted from OSH in NY. She has PMH o...,5222,466,...,"[[O2 SAT, a persistent tracheoesophageal fistu...",Pt was admitted to the SICU for bronchoscopy a...,1.0,0.666667,13.1313,3.1949,10.1408,0.0237,0.241552,0.375646
264,57507108029,Sinus rhythm. {{Diffuse ST segment elevation}}...,29 yo M w no significant PMH presented with pl...,6223,1388,7611,0.620690,Sinus rhythm. Diffuse ST segment elevation and...,2083,368,...,"[[left ventricular function, a moderate sized ...",The patient was admitted to the hospital for f...,1.0,0.909091,23.2558,6.7568,16.2791,10.0334,0.381379,0.465292


In [8]:
result_set.columns

Index(['id', 'source', 'target', 'len_source', 'len_target', 'len_total',
       'label_entity_coverage_rate', 'source_orig', 'len_src_tok',
       'len_tgt_tok', 'len_total_tok', 'oracle_salient_ESGs', 'output', 'SGR',
       'HR', 'rouge-1%', 'rouge-2%', 'rouge-l%', 'bleu-4', 'bs_f1', 'bs_p'],
      dtype='object')

In [9]:
metric_keys = ['SGR', 'HR', 'rouge-1%', 'rouge-2%', 'rouge-l%', 'bleu-4', 'bs_f1', 'bs_p']
for k in metric_keys:
    print(k, np.mean(result_set[k]))

SGR 0.6875492336969925
HR 0.7176862018233082
rouge-1% 20.693469548872184
rouge-2% 6.444496992481203
rouge-l% 12.235481203007518
bleu-4 11.868058270676691
bs_f1 0.3197358286612782
bs_p 0.38628581421240604


In [11]:
result_set.to_json('all_merged_for_case_study.jsonl', lines=True, orient='records')